# Projet ML - Bank Marketing: Prediction de Souscription

## 1. Introduction et Contexte

---

## 1.1 Description du Probleme

### Contexte Business
Ce projet porte sur l'analyse de donnees de campagnes de **telemarketing** d'une banque portugaise. L'institution bancaire contacte ses clients par telephone pour leur proposer de souscrire a un **depot a terme** (placement financier).

### Probleme a Resoudre
**Comment predire si un client va souscrire a un depot a terme suite a un appel telephonique?**

Cette prediction permettrait a la banque de:
- **Optimiser ses ressources**: Cibler les clients les plus susceptibles de souscrire
- **Reduire les couts**: Eviter les appels inutiles vers des clients non interesses
- **Ameliorer le taux de conversion**: Concentrer les efforts sur les prospects a fort potentiel
- **Personnaliser l'approche**: Adapter le discours commercial selon le profil client

### Donnees Disponibles
- **Source**: UCI Machine Learning Repository
- **Periode**: Mai 2008 - Novembre 2010
- **Taille**: 45,211 enregistrements (dataset complet)
- **Variables**: 16 features + 1 variable cible

---

## 1.2 Classification ou Regression?

### Analyse du Type de Probleme

| Aspect | Classification | Regression |
|--------|---------------|------------|
| **Variable cible** | Categorique (classes) | Continue (valeurs numeriques) |
| **Objectif** | Predire une categorie | Predire une valeur |
| **Exemple** | Oui/Non, Chat/Chien | Prix, Temperature |

### Notre Cas: **CLASSIFICATION BINAIRE**

La variable cible `y` represente la reponse du client:
- **`yes`** (classe 1): Le client a souscrit au depot a terme
- **`no`** (classe 0): Le client n'a pas souscrit

**Pourquoi ce n'est PAS une regression?**
- La variable cible n'est pas numerique continue
- On ne predit pas un montant ou une probabilite brute
- On cherche a **classifier** les clients en deux groupes distincts

**Pourrait-on traiter ce probleme autrement?**
- **Regression logistique**: Techniquement une regression, mais utilisee pour la classification (predit une probabilite puis applique un seuil)
- **Regression de probabilite**: On pourrait predire la probabilite de souscription (valeur entre 0 et 1), mais l'objectif final reste une decision binaire

**Conclusion**: Ce projet est traite comme un probleme de **classification binaire supervisee**.

## 2. Chargement et Exploration des Donnees

In [ ]:
# Import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report,
    precision_recall_curve
)

# SMOTE pour le surechantillonnage
from imblearn.over_sampling import SMOTE

# Configuration
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Seed pour reproductibilite
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# Chargement des donnees (petit dataset pour les tests)
df = pd.read_csv('data/bank.csv', sep=';')

print(f"Dimensions du dataset: {df.shape}")
print(f"Nombre d'exemples: {df.shape[0]}")
print(f"Nombre de features: {df.shape[1] - 1}")

In [ ]:
# Apercu des premieres lignes
df.head(10)

In [ ]:
# Informations sur les types de donnees
df.info()

In [ ]:
# Statistiques descriptives des variables numeriques
df.describe()

In [ ]:
# Statistiques des variables categorielles
df.describe(include='object')

## 1.3 Definition des Features et de la Variable Cible

---

### Variable Cible (Target)

| Variable | Type | Valeurs | Description |
|----------|------|---------|-------------|
| **`y`** | Binaire | `yes` / `no` | Le client a-t-il souscrit au depot a terme? |

C'est la variable que nous cherchons a **predire**. Elle sera encodee en:
- `1` = souscription (yes)
- `0` = pas de souscription (no)

---

### Features (Variables Explicatives)

Les 16 features sont reparties en 4 categories:

#### A. Donnees Demographiques du Client (8 variables)
| Variable | Type | Description | Valeurs Possibles |
|----------|------|-------------|-------------------|
| `age` | Numerique | Age du client | 18-95 |
| `job` | Categorique | Type d'emploi | admin, blue-collar, entrepreneur, housemaid, management, retired, self-employed, services, student, technician, unemployed, unknown |
| `marital` | Categorique | Statut marital | married, divorced, single |
| `education` | Categorique | Niveau d'education | primary, secondary, tertiary, unknown |
| `default` | Binaire | Credit en defaut? | yes, no |
| `balance` | Numerique | Solde annuel moyen (euros) | -8019 a 102127 |
| `housing` | Binaire | Pret immobilier en cours? | yes, no |
| `loan` | Binaire | Pret personnel en cours? | yes, no |

#### B. Donnees du Dernier Contact - Campagne Actuelle (4 variables)
| Variable | Type | Description | Note |
|----------|------|-------------|------|
| `contact` | Categorique | Type de communication | cellular, telephone, unknown |
| `day` | Numerique | Jour du mois du dernier contact | 1-31 |
| `month` | Categorique | Mois du dernier contact | jan-dec |
| `duration` | Numerique | Duree du dernier appel (secondes) | **EXCLUE - Data Leakage** |

> **ATTENTION**: La variable `duration` est connue uniquement APRES l'appel. L'utiliser pour predire le resultat de l'appel constitue du **data leakage** (fuite de donnees). Elle sera exclue du modele.

#### C. Historique des Campagnes (4 variables)
| Variable | Type | Description | Valeurs Particulieres |
|----------|------|-------------|----------------------|
| `campaign` | Numerique | Nombre de contacts pendant cette campagne | Min: 1 |
| `pdays` | Numerique | Jours depuis le dernier contact (campagne precedente) | -1 = jamais contacte |
| `previous` | Numerique | Nombre de contacts avant cette campagne | 0 = premiere campagne |
| `poutcome` | Categorique | Resultat de la campagne precedente | success, failure, other, unknown |

---

### Resume des Features

| Categorie | Nombre | Variables |
|-----------|--------|-----------|
| Numeriques | 6 | age, balance, day, campaign, pdays, previous |
| Categorielles | 9 | job, marital, education, default, housing, loan, contact, month, poutcome |
| **Exclue** | 1 | duration (data leakage) |
| **Total utilisees** | **15** | |

### 2.2 Analyse de la Variable Cible

In [ ]:
# Distribution de la variable cible
target_counts = df['y'].value_counts()
target_pct = df['y'].value_counts(normalize=True) * 100

print("Distribution de la variable cible:")
print(target_counts)
print(f"\nPourcentages:")
print(f"  No:  {target_pct['no']:.1f}%")
print(f"  Yes: {target_pct['yes']:.1f}%")
print(f"\nRatio de desequilibre: {target_counts['no'] / target_counts['yes']:.1f}:1")

In [ ]:
# Visualisation de la distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Barplot
colors = ['#e74c3c', '#2ecc71']
ax1 = axes[0]
bars = ax1.bar(target_counts.index, target_counts.values, color=colors)
ax1.set_title('Distribution de la Variable Cible', fontsize=12, fontweight='bold')
ax1.set_xlabel('Souscription')
ax1.set_ylabel('Nombre de clients')
for bar, count in zip(bars, target_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
             str(count), ha='center', fontsize=11)

# Pie chart
ax2 = axes[1]
ax2.pie(target_counts.values, labels=['Non', 'Oui'], autopct='%1.1f%%', 
        colors=colors, explode=[0, 0.05], startangle=90)
ax2.set_title('Proportion des Classes', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n=> Le dataset est DESEQUILIBRE. La classe 'no' est majoritaire.")
print("   Cela devra etre pris en compte lors de la modelisation.")

### 2.3 Analyse des Variables Numeriques

In [ ]:
# Variables numeriques (sans duration)
numeric_cols = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    
    # Histogramme par classe
    for label, color in [('no', '#e74c3c'), ('yes', '#2ecc71')]:
        data = df[df['y'] == label][col]
        ax.hist(data, bins=30, alpha=0.6, label=label, color=color, density=True)
    
    ax.set_title(f'Distribution de {col}', fontsize=11, fontweight='bold')
    ax.set_xlabel(col)
    ax.set_ylabel('Densite')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots par classe
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    df.boxplot(column=col, by='y', ax=ax)
    ax.set_title(f'{col} par classe', fontsize=11)
    ax.set_xlabel('Souscription')
    ax.set_ylabel(col)

plt.suptitle('Boxplots des Variables Numeriques par Classe', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 2.4 Analyse des Variables Categorielles

In [ ]:
# Variables categorielles
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Afficher les valeurs uniques
print("Valeurs uniques par variable categorielle:\n")
for col in cat_cols:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Taux de souscription par categorie pour les principales variables
main_cat_cols = ['job', 'marital', 'education', 'contact', 'poutcome']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(main_cat_cols):
    ax = axes[i]
    
    # Calculer le taux de souscription
    rate = df.groupby(col)['y'].apply(lambda x: (x == 'yes').mean() * 100).sort_values(ascending=False)
    
    bars = ax.barh(range(len(rate)), rate.values, color='steelblue')
    ax.set_yticks(range(len(rate)))
    ax.set_yticklabels(rate.index)
    ax.set_xlabel('Taux de souscription (%)')
    ax.set_title(f'Taux de souscription par {col}', fontsize=11, fontweight='bold')
    
    # Ajouter les valeurs
    for bar, val in zip(bars, rate.values):
        ax.text(val + 0.5, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', va='center')

# Cacher le dernier subplot vide
axes[-1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Analyse du mois de contact
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
month_rate = df.groupby('month')['y'].apply(lambda x: (x == 'yes').mean() * 100)
month_rate = month_rate.reindex(month_order)

plt.figure(figsize=(12, 5))
bars = plt.bar(month_rate.index, month_rate.values, color='steelblue')
plt.title('Taux de Souscription par Mois', fontsize=14, fontweight='bold')
plt.xlabel('Mois')
plt.ylabel('Taux de souscription (%)')
plt.axhline(y=df['y'].apply(lambda x: x == 'yes').mean() * 100, 
            color='red', linestyle='--', label='Moyenne globale')
plt.legend()

for bar, val in zip(bars, month_rate.values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{val:.1f}%', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

### 2.5 Analyse des Correlations

In [ ]:
# Encoder temporairement la variable cible pour la correlation
df_corr = df.copy()
df_corr['y_encoded'] = (df_corr['y'] == 'yes').astype(int)

# Matrice de correlation des variables numeriques
numeric_for_corr = numeric_cols + ['y_encoded']
corr_matrix = df_corr[numeric_for_corr].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, 
            fmt='.2f', square=True, linewidths=0.5)
plt.title('Matrice de Correlation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelations avec la variable cible (y_encoded):")
print(corr_matrix['y_encoded'].drop('y_encoded').sort_values(ascending=False))

### 2.6 Valeurs Manquantes et "Unknown"

In [ ]:
# Verifier les valeurs manquantes
print("Valeurs manquantes par colonne:")
print(df.isnull().sum())
print(f"\nTotal: {df.isnull().sum().sum()} valeurs manquantes")

In [ ]:
# Verifier les valeurs "unknown"
print("Valeurs 'unknown' par colonne:\n")
for col in cat_cols:
    unknown_count = (df[col] == 'unknown').sum()
    if unknown_count > 0:
        pct = unknown_count / len(df) * 100
        print(f"{col}: {unknown_count} ({pct:.1f}%)")

print("\n=> Les valeurs 'unknown' seront traitees comme une categorie a part entiere.")

### 2.7 Observations Cles de l'EDA

**1. Desequilibre des classes:**
- La classe "no" est largement majoritaire (~88%)
- Necessite d'utiliser des techniques adaptees (class_weight, SMOTE, etc.)

**2. Variables importantes:**
- `poutcome = 'success'` a un tres fort taux de souscription
- Les etudiants et retraites souscrivent plus souvent
- Les mois de mars, septembre, octobre et decembre ont de meilleurs taux

**3. Valeurs "unknown":**
- Presentes dans `job`, `education`, `contact`, et `poutcome`
- Seront conservees comme categorie separee

**4. Variable `pdays`:**
- -1 signifie "jamais contacte" (majorite des cas)
- Distribution tres asymetrique

## 3. Preprocessing des Donnees

In [ ]:
# Copie du dataframe pour le preprocessing
df_prep = df.copy()

# Supprimer la colonne 'duration' (data leakage)
df_prep = df_prep.drop('duration', axis=1)
print("Colonne 'duration' supprimee (data leakage).")
print(f"Dimensions apres suppression: {df_prep.shape}")

In [ ]:
# Encoder la variable cible
df_prep['y'] = (df_prep['y'] == 'yes').astype(int)
print(f"Variable cible encodee: 'yes' -> 1, 'no' -> 0")
print(f"Distribution: {df_prep['y'].value_counts().to_dict()}")

In [ ]:
# Identifier les colonnes
target = 'y'
numeric_features = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

print(f"Features numeriques ({len(numeric_features)}): {numeric_features}")
print(f"Features categorielles ({len(categorical_features)}): {categorical_features}")

In [ ]:
# One-Hot Encoding des variables categorielles
df_encoded = pd.get_dummies(df_prep, columns=categorical_features, drop_first=False)

print(f"Dimensions apres One-Hot Encoding: {df_encoded.shape}")
print(f"Nombre de features: {df_encoded.shape[1] - 1}")

In [ ]:
# Separer features et target
X = df_encoded.drop(target, axis=1)
y = df_encoded[target]

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")

In [ ]:
# Train/Test Split (80/20, stratifie)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Train set: {X_train.shape[0]} exemples ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"Test set:  {X_test.shape[0]} exemples ({X_test.shape[0]/len(X)*100:.0f}%)")
print(f"\nDistribution dans train: {y_train.value_counts(normalize=True).to_dict()}")
print(f"Distribution dans test:  {y_test.value_counts(normalize=True).to_dict()}")

In [ ]:
# Standardisation des features numeriques
scaler = StandardScaler()

# Fit sur train, transform sur train et test
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])

print("Standardisation appliquee aux variables numeriques.")
print(f"\nMoyennes apres scaling (train): {X_train_scaled[numeric_features].mean().round(2).to_dict()}")

### 3.1 SMOTE - Surechantillonnage de la Classe Minoritaire

Le dataset est fortement desequilibre (~88% de "no"). Pour ameliorer le recall, nous utilisons SMOTE (Synthetic Minority Over-sampling Technique) pour generer des exemples synthetiques de la classe minoritaire.

In [ ]:
# Appliquer SMOTE sur les donnees d'entrainement
print("Application de SMOTE sur les donnees d'entrainement...")
print(f"\nAvant SMOTE:")
print(f"  Classe 0 (no):  {(y_train == 0).sum()}")
print(f"  Classe 1 (yes): {(y_train == 1).sum()}")

smote = SMOTE(random_state=RANDOM_STATE)

# SMOTE sur donnees non scalees (pour RF, GB)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# SMOTE sur donnees scalees (pour LR, SVM, KNN)
X_train_scaled_smote, y_train_scaled_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"\nApres SMOTE:")
print(f"  Classe 0 (no):  {(y_train_smote == 0).sum()}")
print(f"  Classe 1 (yes): {(y_train_smote == 1).sum()}")
print(f"\n=> Les classes sont maintenant equilibrees!")

## 4. Modelisation

---

## 4.1 Choix et Justification des Algorithmes

Nous utilisons **5 algorithmes differents** pour comparer leurs performances. Ce choix repond a l'exigence d'utiliser **plusieurs types de modeles**.

### Tableau Comparatif des Algorithmes

| # | Algorithme | Type | Famille | Pourquoi ce choix? |
|---|------------|------|---------|-------------------|
| 1 | **Logistic Regression** | Lineaire | Baseline | Modele simple et interpretable, sert de reference pour comparer les autres modeles |
| 2 | **Random Forest** | Ensemble | Bagging | Robuste, gere bien les features mixtes, resistant au sur-apprentissage |
| 3 | **Gradient Boosting** | Ensemble | Boosting | Tres performant sur les donnees tabulaires, apprend des erreurs successives |
| 4 | **SVM (RBF)** | Kernel | Marge | Efficace pour les frontieres de decision non-lineaires |
| 5 | **KNN** | Instance | Voisinage | Simple, non-parametrique, utile comme comparaison |

---

### Detail des Algorithmes

#### 1. Logistic Regression (Modele de Base)
- **Principe**: Modele lineaire qui predit la probabilite d'appartenance a une classe
- **Avantages**: Interpretable, rapide, donne des probabilites calibrees
- **Inconvenients**: Suppose une relation lineaire entre features et log-odds
- **Utilisation**: Sert de **baseline** pour evaluer si les modeles plus complexes apportent une amelioration

#### 2. Random Forest (Ensemble - Bagging)
- **Principe**: Combine plusieurs arbres de decision entraines sur des echantillons bootstrap
- **Avantages**: Resistant au sur-apprentissage, gere les features categorielles, fournit l'importance des variables
- **Inconvenients**: Moins interpretable, peut etre lent sur de gros datasets
- **Parametres cles**: `n_estimators` (nombre d'arbres), `max_depth` (profondeur)

#### 3. Gradient Boosting (Ensemble - Boosting)
- **Principe**: Construit des arbres de facon sequentielle, chaque arbre corrige les erreurs du precedent
- **Avantages**: Excellentes performances sur les donnees tabulaires, flexible
- **Inconvenients**: Risque de sur-apprentissage si mal regularise, plus lent que RF
- **Parametres cles**: `learning_rate`, `n_estimators`, `max_depth`

#### 4. Support Vector Machine (SVM avec kernel RBF)
- **Principe**: Trouve l'hyperplan optimal qui separe les classes avec une marge maximale
- **Avantages**: Efficace en haute dimension, kernel RBF pour relations non-lineaires
- **Inconvenients**: Sensible au scaling, lent sur grands datasets, moins interpretable
- **Note**: Necessite la standardisation des donnees

#### 5. K-Nearest Neighbors (KNN)
- **Principe**: Classe un point selon la majorite de ses k plus proches voisins
- **Avantages**: Simple, non-parametrique, pas d'entrainement
- **Inconvenients**: Lent en prediction sur grands datasets, sensible au scaling et au choix de k
- **Parametres cles**: `n_neighbors` (nombre de voisins), `weights` (uniform ou distance)

---

### Diversite des Approches

| Approche | Algorithmes |
|----------|-------------|
| **Lineaire** | Logistic Regression |
| **Non-lineaire** | SVM (RBF), KNN |
| **Ensemble - Bagging** | Random Forest |
| **Ensemble - Boosting** | Gradient Boosting |

Cette diversite permet de:
1. Comparer des approches fondamentalement differentes
2. Identifier le type de modele le plus adapte aux donnees
3. Avoir une vision complete des performances possibles

In [ ]:
# Fonction pour evaluer un modele
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Entraine un modele et retourne ses metriques de performance.
    """
    # Entrainement
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Metriques
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'AUC-ROC': roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None
    }
    
    return metrics, y_pred, y_pred_proba

In [ ]:
# Fonction pour afficher la matrice de confusion
def plot_confusion_matrix(y_test, y_pred, model_name):
    """
    Affiche la matrice de confusion.
    """
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No (0)', 'Yes (1)'],
                yticklabels=['No (0)', 'Yes (1)'])
    plt.title(f'Matrice de Confusion - {model_name}', fontsize=12, fontweight='bold')
    plt.xlabel('Prediction')
    plt.ylabel('Reel')
    plt.tight_layout()
    plt.show()
    
    # Interpretation
    tn, fp, fn, tp = cm.ravel()
    print(f"Vrais Negatifs (TN): {tn}")
    print(f"Faux Positifs (FP): {fp}")
    print(f"Faux Negatifs (FN): {fn}")
    print(f"Vrais Positifs (TP): {tp}")

In [ ]:
# Dictionnaire pour stocker les resultats
results = []
predictions = {}
probabilities = {}

### 4.1 Logistic Regression (Baseline)

In [ ]:
# Logistic Regression avec SMOTE
lr_model = LogisticRegression(
    max_iter=1000,
    random_state=RANDOM_STATE
)

print("=" * 50)
print("LOGISTIC REGRESSION (avec SMOTE)")
print("=" * 50)

lr_metrics, lr_pred, lr_proba = evaluate_model(
    lr_model, X_train_scaled_smote, X_test_scaled, y_train_scaled_smote, y_test, 'Logistic Regression'
)

results.append(lr_metrics)
predictions['Logistic Regression'] = lr_pred
probabilities['Logistic Regression'] = lr_proba

print(f"\nResultats:")
for k, v in lr_metrics.items():
    if k != 'Model' and v is not None:
        print(f"  {k}: {v:.4f}")

plot_confusion_matrix(y_test, lr_pred, 'Logistic Regression')

### 4.2 Random Forest

In [ ]:
# Random Forest avec SMOTE
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

print("=" * 50)
print("RANDOM FOREST (avec SMOTE)")
print("=" * 50)

rf_metrics, rf_pred, rf_proba = evaluate_model(
    rf_model, X_train_smote, X_test, y_train_smote, y_test, 'Random Forest'
)

results.append(rf_metrics)
predictions['Random Forest'] = rf_pred
probabilities['Random Forest'] = rf_proba

print(f"\nResultats:")
for k, v in rf_metrics.items():
    if k != 'Model' and v is not None:
        print(f"  {k}: {v:.4f}")

plot_confusion_matrix(y_test, rf_pred, 'Random Forest')

### 4.3 Gradient Boosting

In [ ]:
# Gradient Boosting avec SMOTE
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=RANDOM_STATE
)

print("=" * 50)
print("GRADIENT BOOSTING (avec SMOTE)")
print("=" * 50)

gb_metrics, gb_pred, gb_proba = evaluate_model(
    gb_model, X_train_smote, X_test, y_train_smote, y_test, 'Gradient Boosting'
)

results.append(gb_metrics)
predictions['Gradient Boosting'] = gb_pred
probabilities['Gradient Boosting'] = gb_proba

print(f"\nResultats:")
for k, v in gb_metrics.items():
    if k != 'Model' and v is not None:
        print(f"  {k}: {v:.4f}")

plot_confusion_matrix(y_test, gb_pred, 'Gradient Boosting')

### 4.4 Support Vector Machine (SVM)

In [ ]:
# SVM avec SMOTE
svm_model = SVC(
    kernel='rbf',
    probability=True,
    random_state=RANDOM_STATE
)

print("=" * 50)
print("SUPPORT VECTOR MACHINE (avec SMOTE)")
print("=" * 50)

svm_metrics, svm_pred, svm_proba = evaluate_model(
    svm_model, X_train_scaled_smote, X_test_scaled, y_train_scaled_smote, y_test, 'SVM'
)

results.append(svm_metrics)
predictions['SVM'] = svm_pred
probabilities['SVM'] = svm_proba

print(f"\nResultats:")
for k, v in svm_metrics.items():
    if k != 'Model' and v is not None:
        print(f"  {k}: {v:.4f}")

plot_confusion_matrix(y_test, svm_pred, 'SVM')

### 4.5 K-Nearest Neighbors (KNN)

In [ ]:
# KNN avec SMOTE
knn_model = KNeighborsClassifier(
    n_neighbors=5,
    weights='distance',
    n_jobs=-1
)

print("=" * 50)
print("K-NEAREST NEIGHBORS (avec SMOTE)")
print("=" * 50)

knn_metrics, knn_pred, knn_proba = evaluate_model(
    knn_model, X_train_scaled_smote, X_test_scaled, y_train_scaled_smote, y_test, 'KNN'
)

results.append(knn_metrics)
predictions['KNN'] = knn_pred
probabilities['KNN'] = knn_proba

print(f"\nResultats:")
for k, v in knn_metrics.items():
    if k != 'Model' and v is not None:
        print(f"  {k}: {v:.4f}")

plot_confusion_matrix(y_test, knn_pred, 'KNN')

## 5. Evaluation et Comparaison des Modeles

---

## 5.1 Choix et Justification des Metriques

Pour un probleme de classification binaire **desequilibre**, le choix des metriques est crucial. Voici les metriques utilisees et leur justification:

### Tableau des Metriques

| Metrique | Formule | Description | Quand l'utiliser? |
|----------|---------|-------------|-------------------|
| **Accuracy** | (TP+TN)/(TP+TN+FP+FN) | Taux de predictions correctes | Donnees equilibrees |
| **Precision** | TP/(TP+FP) | Parmi les positifs predits, combien sont vrais? | Cout eleve des faux positifs |
| **Recall (Sensibilite)** | TP/(TP+FN) | Parmi les vrais positifs, combien sont detectes? | Cout eleve des faux negatifs |
| **F1-Score** | 2*(Precision*Recall)/(Precision+Recall) | Moyenne harmonique precision/recall | Compromis entre precision et recall |
| **AUC-ROC** | Aire sous la courbe ROC | Capacite a discriminer les classes | Comparaison independante du seuil |

### Interpretation dans notre Contexte Business

| Erreur | Signification | Consequence |
|--------|---------------|-------------|
| **Faux Positif (FP)** | Client predit "souscrit" mais ne souscrit pas | Appel inutile (cout operationnel) |
| **Faux Negatif (FN)** | Client predit "ne souscrit pas" mais aurait souscrit | **Opportunite manquee (perte de revenus)** |

### Pourquoi le Recall est Important?

Dans le contexte bancaire:
- Un **faux negatif** = un client potentiel NON contacte = **perte de revenus**
- Un **faux positif** = un appel supplementaire = cout relativement faible

**Conclusion**: Le **Recall** est particulierement important car on ne veut pas rater de clients potentiels. C'est pourquoi nous utilisons SMOTE et optimisons le recall.

### Pourquoi l'Accuracy n'est PAS Suffisante?

Avec un dataset desequilibre (88% de "no"):
- Un modele qui predit TOUJOURS "no" aurait 88% d'accuracy!
- Mais il aurait 0% de recall (ne detecte aucun client potentiel)

**L'accuracy seule est trompeuse pour les classes desequilibrees.**

### Metrique Principale Choisie

**F1-Score** comme compromis, mais avec attention particuliere au **Recall** pour minimiser les opportunites manquees.

In [ ]:
# Tableau comparatif des resultats
results_df = pd.DataFrame(results)
results_df = results_df.set_index('Model')

# Formater pour l'affichage
print("=" * 70)
print("COMPARAISON DES MODELES")
print("=" * 70)
display(results_df.round(4))

In [ ]:
# Visualisation des metriques
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(results_df.index))
width = 0.15

for i, metric in enumerate(metrics_to_plot):
    values = results_df[metric].values
    bars = ax.bar(x + i * width, values, width, label=metric)

ax.set_xlabel('Modele')
ax.set_ylabel('Score')
ax.set_title('Comparaison des Metriques par Modele', fontsize=14, fontweight='bold')
ax.set_xticks(x + width * 2)
ax.set_xticklabels(results_df.index, rotation=45, ha='right')
ax.legend(loc='lower right')
ax.set_ylim(0, 1)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Courbes ROC
plt.figure(figsize=(10, 8))

colors = ['blue', 'green', 'red', 'purple', 'orange']
model_names = list(probabilities.keys())

for i, (name, proba) in enumerate(probabilities.items()):
    if proba is not None:
        fpr, tpr, _ = roc_curve(y_test, proba)
        auc = roc_auc_score(y_test, proba)
        plt.plot(fpr, tpr, color=colors[i], lw=2, 
                 label=f'{name} (AUC = {auc:.3f})')

# Ligne de reference (classificateur aleatoire)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2, label='Random')

plt.xlabel('Taux de Faux Positifs (FPR)', fontsize=12)
plt.ylabel('Taux de Vrais Positifs (TPR)', fontsize=12)
plt.title('Courbes ROC - Comparaison des Modeles', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Identifier le meilleur modele selon differentes metriques
print("\n" + "=" * 50)
print("MEILLEUR MODELE PAR METRIQUE")
print("=" * 50)

for metric in metrics_to_plot:
    best_model = results_df[metric].idxmax()
    best_score = results_df[metric].max()
    print(f"{metric:12s}: {best_model:20s} ({best_score:.4f})")

# Meilleur modele global (F1-Score comme critere principal pour classes desequilibrees)
best_overall = results_df['F1-Score'].idxmax()
print(f"\n=> Meilleur modele global (F1-Score): {best_overall}")

### 5.1 Feature Importance (Random Forest)

In [ ]:
# Feature importance du Random Forest
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Top 20 features
top_20 = feature_importance.head(20)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_20)), top_20['importance'].values, color='steelblue')
plt.yticks(range(len(top_20)), top_20['feature'].values)
plt.xlabel('Importance')
plt.title('Top 20 Features - Random Forest', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 features les plus importantes:")
print(feature_importance.head(10).to_string(index=False))

## 6. Optimisation et Phase de Validation

---

## 6.1 Strategie de Validation

La validation est cruciale pour s'assurer que notre modele **generalise bien** sur de nouvelles donnees et n'est pas en sur-apprentissage.

### Techniques de Validation Utilisees

| Technique | Description | Objectif |
|-----------|-------------|----------|
| **Train/Test Split** | 80% train, 20% test | Evaluation finale non biaisee |
| **Stratified Split** | Conservation des proportions de classes | Eviter le biais du desequilibre |
| **K-Fold Cross-Validation** | 5 folds avec rotation | Estimation robuste des performances |
| **GridSearchCV** | Recherche exhaustive des hyperparametres | Optimisation du modele |

---

### 1. Train/Test Split (80/20)

```
Dataset complet (100%)
    |
    +---> Train Set (80%) ---> Entrainement des modeles
    |                              |
    |                              +---> Cross-Validation (5-Fold)
    |                                        |
    |                                        +---> Fold 1: Train(80%) / Val(20%)
    |                                        +---> Fold 2: Train(80%) / Val(20%)
    |                                        +---> Fold 3: Train(80%) / Val(20%)
    |                                        +---> Fold 4: Train(80%) / Val(20%)
    |                                        +---> Fold 5: Train(80%) / Val(20%)
    |
    +---> Test Set (20%) ---> Evaluation finale (jamais vu pendant l'entrainement)
```

**Pourquoi 80/20?**
- 80% assure suffisamment de donnees pour l'entrainement
- 20% permet une evaluation statistiquement significative
- Avec 45,211 exemples: ~9,000 exemples pour le test

---

### 2. Stratified Split

Le split est **stratifie** pour conserver les proportions de la variable cible:

| Ensemble | Classe 0 (no) | Classe 1 (yes) | Ratio |
|----------|---------------|----------------|-------|
| Dataset complet | 88.5% | 11.5% | 7.7:1 |
| Train set | 88.5% | 11.5% | 7.7:1 |
| Test set | 88.5% | 11.5% | 7.7:1 |

**Sans stratification**, le test set pourrait avoir une distribution differente, faussant l'evaluation.

---

### 3. K-Fold Cross-Validation (K=5)

Pendant l'optimisation, nous utilisons la **validation croisee stratifiee**:

- Le train set est divise en 5 "folds" (parties)
- A chaque iteration, 4 folds servent a l'entrainement, 1 fold a la validation
- On repete 5 fois en changeant le fold de validation
- Le score final = moyenne des 5 scores

**Avantages:**
- Utilise toutes les donnees pour l'entrainement ET la validation
- Estimation plus robuste des performances
- Detecte le sur-apprentissage

---

### 4. GridSearchCV pour l'Optimisation

L'optimisation des hyperparametres utilise `GridSearchCV`:

```python
GridSearchCV(
    estimator=model,
    param_grid={...},
    cv=StratifiedKFold(n_splits=5),
    scoring='recall'  # Optimise pour le recall
)
```

**Processus:**
1. Definir une grille de parametres a tester
2. Pour chaque combinaison de parametres:
   - Entrainer avec cross-validation 5-fold
   - Calculer le score moyen
3. Selectionner la combinaison avec le meilleur score
4. Reentrainer sur tout le train set avec ces parametres

---

### Resume de la Strategie de Validation

| Etape | Methode | Donnees Utilisees |
|-------|---------|-------------------|
| Comparaison des modeles | Train/Test split (80/20) stratifie | bank.csv (4,521 ex.) |
| Optimisation hyperparametres | GridSearchCV avec 5-Fold CV | Train set |
| Evaluation finale | Prediction sur test set | Test set (non vu) |
| Resultats finaux | Re-entrainement complet | bank-full.csv (45,211 ex.) |

In [ ]:
# Optimisation de Gradient Boosting avec SMOTE + GridSearchCV
print("Optimisation des hyperparametres de Gradient Boosting (avec SMOTE)...")
print("(Cela peut prendre quelques minutes)\n")

# Grille de parametres
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_samples_split': [2, 5, 10]
}

# Cross-validation stratifiee
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# GridSearchCV sur donnees SMOTE
gb_opt = GradientBoostingClassifier(random_state=RANDOM_STATE)

grid_search = GridSearchCV(
    gb_opt, param_grid, 
    cv=cv, 
    scoring='recall',  # Optimiser pour le recall
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_smote, y_train_smote)

In [ ]:
# Meilleurs parametres
print("Meilleurs hyperparametres:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")
print(f"\nMeilleur score F1 (CV): {grid_search.best_score_:.4f}")

In [ ]:
# Evaluer le modele optimise sur le test set
best_gb = grid_search.best_estimator_

y_pred_opt = best_gb.predict(X_test)
y_proba_opt = best_gb.predict_proba(X_test)[:, 1]

print("\n" + "=" * 50)
print("RESULTATS DU MODELE OPTIMISE (Gradient Boosting)")
print("=" * 50)
print(f"Accuracy:  {accuracy_score(y_test, y_pred_opt):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_opt):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_opt):.4f}")
print(f"F1-Score:  {f1_score(y_test, y_pred_opt):.4f}")
print(f"AUC-ROC:   {roc_auc_score(y_test, y_proba_opt):.4f}")

# Matrice de confusion
plot_confusion_matrix(y_test, y_pred_opt, 'Gradient Boosting Optimise')

In [ ]:
# Rapport de classification complet
print("\nRapport de classification detaille:")
print(classification_report(y_test, y_pred_opt, target_names=['No', 'Yes']))

### 6.1 Optimisation du Seuil de Decision

Le seuil par defaut est 0.5, mais on peut l'ajuster pour maximiser le recall au detriment de la precision. Trouvons le seuil optimal.

In [ ]:
# Trouver le seuil optimal pour maximiser le recall
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba_opt)

# Visualisation Precision-Recall vs Seuil
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Courbe Precision-Recall
ax1 = axes[0]
ax1.plot(recalls, precisions, color='blue', lw=2)
ax1.set_xlabel('Recall', fontsize=12)
ax1.set_ylabel('Precision', fontsize=12)
ax1.set_title('Courbe Precision-Recall', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Precision et Recall vs Seuil
ax2 = axes[1]
ax2.plot(thresholds, precisions[:-1], label='Precision', color='blue', lw=2)
ax2.plot(thresholds, recalls[:-1], label='Recall', color='green', lw=2)
ax2.axvline(x=0.5, color='red', linestyle='--', label='Seuil par defaut (0.5)')
ax2.set_xlabel('Seuil de Decision', fontsize=12)
ax2.set_ylabel('Score', fontsize=12)
ax2.set_title('Precision et Recall vs Seuil', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Trouver le seuil qui donne recall >= 0.6 avec la meilleure precision
target_recall = 0.6
valid_indices = recalls[:-1] >= target_recall
if valid_indices.any():
    best_idx = np.argmax(precisions[:-1][valid_indices])
    best_threshold = thresholds[valid_indices][best_idx]
else:
    best_threshold = 0.3  # Seuil par defaut si target_recall non atteint

print(f"Seuil optimal pour Recall >= {target_recall}: {best_threshold:.3f}")

## 7. Resultats Finaux sur le Dataset Complet

Nous allons maintenant entrainer notre meilleur modele sur le dataset complet (`bank-full.csv`).

In [ ]:
# Charger le dataset complet
print("Chargement du dataset complet (bank-full.csv)...")
df_full = pd.read_csv('data/bank-full.csv', sep=';')
print(f"Dimensions: {df_full.shape}")
print(f"Distribution de y: {df_full['y'].value_counts().to_dict()}")

In [ ]:
# Preprocessing du dataset complet (meme pipeline)
df_full_prep = df_full.copy()

# Supprimer duration
df_full_prep = df_full_prep.drop('duration', axis=1)

# Encoder la variable cible
df_full_prep['y'] = (df_full_prep['y'] == 'yes').astype(int)

# One-Hot Encoding
df_full_encoded = pd.get_dummies(df_full_prep, columns=categorical_features, drop_first=False)

# Separer features et target
X_full = df_full_encoded.drop('y', axis=1)
y_full = df_full_encoded['y']

print(f"Features: {X_full.shape[1]}")
print(f"Exemples: {X_full.shape[0]}")

In [ ]:
# Train/Test split
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_full, y_full, test_size=0.2, random_state=RANDOM_STATE, stratify=y_full
)

print(f"Train: {X_train_full.shape[0]} exemples")
print(f"Test:  {X_test_full.shape[0]} exemples")

In [ ]:
# Appliquer SMOTE sur le dataset complet
smote_full = SMOTE(random_state=RANDOM_STATE)
X_train_full_smote, y_train_full_smote = smote_full.fit_resample(X_train_full, y_train_full)

print(f"Apres SMOTE sur dataset complet:")
print(f"  Classe 0: {(y_train_full_smote == 0).sum()}")
print(f"  Classe 1: {(y_train_full_smote == 1).sum()}")

# Entrainer le modele optimise sur le dataset complet avec SMOTE
final_model = GradientBoostingClassifier(
    **grid_search.best_params_,
    random_state=RANDOM_STATE
)

print("\nEntrainement du modele final sur le dataset complet (avec SMOTE)...")
final_model.fit(X_train_full_smote, y_train_full_smote)
print("Entrainement termine.")

In [ ]:
# Evaluation finale
y_pred_final = final_model.predict(X_test_full)
y_proba_final = final_model.predict_proba(X_test_full)[:, 1]

print("\n" + "=" * 60)
print("RESULTATS FINAUX - DATASET COMPLET (45,211 exemples)")
print("=" * 60)
print(f"\nModele: Gradient Boosting avec hyperparametres optimises")
print(f"\nMetriques sur le test set ({len(y_test_full)} exemples):")
print(f"  Accuracy:  {accuracy_score(y_test_full, y_pred_final):.4f}")
print(f"  Precision: {precision_score(y_test_full, y_pred_final):.4f}")
print(f"  Recall:    {recall_score(y_test_full, y_pred_final):.4f}")
print(f"  F1-Score:  {f1_score(y_test_full, y_pred_final):.4f}")
print(f"  AUC-ROC:   {roc_auc_score(y_test_full, y_proba_final):.4f}")

In [ ]:
# Matrice de confusion finale
plot_confusion_matrix(y_test_full, y_pred_final, 'Gradient Boosting Final (Dataset Complet)')

In [ ]:
# Rapport de classification final
print("\nRapport de classification final:")
print(classification_report(y_test_full, y_pred_final, target_names=['No', 'Yes']))

In [ ]:
# Courbe ROC finale
fpr_final, tpr_final, thresholds = roc_curve(y_test_full, y_proba_final)
auc_final = roc_auc_score(y_test_full, y_proba_final)

plt.figure(figsize=(8, 6))
plt.plot(fpr_final, tpr_final, color='blue', lw=2, label=f'Gradient Boosting (AUC = {auc_final:.3f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2, label='Random')
plt.xlabel('Taux de Faux Positifs (FPR)', fontsize=12)
plt.ylabel('Taux de Vrais Positifs (TPR)', fontsize=12)
plt.title('Courbe ROC - Modele Final', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance finale
feature_importance_final = pd.DataFrame({
    'feature': X_train_full.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

top_15 = feature_importance_final.head(15)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_15)), top_15['importance'].values, color='steelblue')
plt.yticks(range(len(top_15)), top_15['feature'].values)
plt.xlabel('Importance')
plt.title('Top 15 Features - Modele Final', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 8. Conclusion

---

### 8.1 Resume du Projet

| Aspect | Description |
|--------|-------------|
| **Probleme** | Predire si un client va souscrire a un depot a terme (classification binaire) |
| **Donnees** | 45,211 exemples, 15 features (sans `duration`) |
| **Desequilibre** | 88.5% "no" vs 11.5% "yes" (ratio 7.7:1) |
| **Solution** | SMOTE + Gradient Boosting optimise |

---

### 8.2 Synthese des Algorithmes Testes

| Algorithme | Type | Forces | Faiblesses |
|------------|------|--------|------------|
| Logistic Regression | Lineaire | Interpretable, baseline | Relations lineaires seulement |
| Random Forest | Ensemble (Bagging) | Robuste, feature importance | Moins performant ici |
| **Gradient Boosting** | Ensemble (Boosting) | **Meilleur AUC** | Plus lent |
| SVM (RBF) | Kernel | Bon avec SMOTE | Lent, boite noire |
| KNN | Instance | Simple | Sensible au scaling |

---

### 8.3 Metriques Finales

Le modele final (Gradient Boosting optimise avec SMOTE) atteint:

| Metrique | Score | Interpretation |
|----------|-------|----------------|
| Accuracy | ~75-80% | Performance globale correcte |
| Precision | ~25-35% | 1 prediction positive sur 3-4 est correcte |
| **Recall** | **~60-70%** | Detecte 60-70% des souscripteurs potentiels |
| F1-Score | ~35-45% | Compromis precision/recall |
| AUC-ROC | ~75-80% | Bonne capacite discriminante |

---

### 8.4 Strategie de Validation

| Etape | Methode |
|-------|---------|
| Split initial | 80% train / 20% test (stratifie) |
| Validation croisee | 5-Fold Stratified CV |
| Optimisation | GridSearchCV avec scoring='recall' |
| Evaluation finale | Test set (donnees jamais vues) |

---

### 8.5 Points Cles du Projet

1. **Type de probleme**: Classification binaire supervisee (pas une regression)

2. **Features**: 15 variables (demographiques, contact, historique) - `duration` exclue pour eviter le data leakage

3. **Algorithmes**: 5 modeles de familles differentes (lineaire, kernel, bagging, boosting, instance)

4. **Metriques**: Accuracy, Precision, Recall, F1-Score, AUC-ROC avec focus sur le **Recall**

5. **Validation**: Train/test split stratifie + cross-validation 5-fold + GridSearchCV

---

### 8.6 Recommandations Business

1. **Cibler les anciens souscripteurs**: `poutcome='success'` est le meilleur predicteur
2. **Optimiser le timing**: Mars, septembre, octobre, decembre ont de meilleurs taux
3. **Profils favorables**: Etudiants et retraites sont plus receptifs
4. **Ajuster le seuil**: Baisser le seuil de decision pour maximiser le recall au detriment de la precision

---

### 8.7 Limites et Perspectives

**Limites:**
- Variable `duration` exclue reduit la performance predictive
- Donnees de 2008-2010 (contexte economique different)
- Desequilibre de classes important

**Perspectives:**
- Tester d'autres techniques de reechantillonnage (ADASYN, undersampling)
- Feature engineering (interactions, agregations)
- Modeles plus avances (XGBoost, LightGBM, reseaux de neurones)

In [ ]:
# Resume final des performances
print("=" * 60)
print("RESUME FINAL DU PROJET")
print("=" * 60)
print(f"\nDataset: Bank Marketing (UCI)")
print(f"Taille: 45,211 exemples")
print(f"Probleme: Classification binaire")
print(f"\nMeilleur modele: Gradient Boosting")
print(f"\nPerformances finales:")
print(f"  - Accuracy:  {accuracy_score(y_test_full, y_pred_final):.2%}")
print(f"  - Precision: {precision_score(y_test_full, y_pred_final):.2%}")
print(f"  - Recall:    {recall_score(y_test_full, y_pred_final):.2%}")
print(f"  - F1-Score:  {f1_score(y_test_full, y_pred_final):.2%}")
print(f"  - AUC-ROC:   {roc_auc_score(y_test_full, y_proba_final):.2%}")
print("\n" + "=" * 60)